In [1]:
%run init_notebookspace.py
from settings import POST_EST_DIR

Current venv: C:\Users\LukasGrahl\Documents\GIT\memoire1\env_macro_rbc
DATA_DIR is existant under: C:\Users\LukasGrahl\Documents\GIT\memoire1\data


In [2]:
from src.utils import get_most_recent_mod_output

import xarray as xr
import numpy as np

import arviz as az
import pandas as pd

In [26]:
def get_xarr_InferenceData(xarr: xr.Dataset, draw: int):
    
    draw = int(draw)
    
    xr_prior = xr.Dataset(
        {
            key: (
                ['chain', 'draw', 'a_dim'],
                np.concatenate(
                    [
                        np.array([0] * draw).reshape(draw, 1),
                        np.array(xarr.sel(parameter=key).new_prior[1:]).reshape(draw, 1),
                        np.array([0] * draw).reshape(draw, 1)
                    ],
                    axis=1
                ).reshape(1, draw, 3)
            )
            for key in xarr.parameter.values
        },
        coords={
            "chain": (["chain"], np.arange(1)),
            "draw": (["draw"], np.arange(draw)),
            "a_dim": (["a_dim"], ["x", "y", "z"])

        }
    )

    xr_post = xr.Dataset(
        {
            key: (
                ['chain', 'draw', 'a_dim'],
                np.concatenate(
                    [
                        np.array([0] * draw).reshape(draw, 1),
                        np.array(xarr.sel(parameter=key).posterior[1:]).reshape(draw, 1),
                        np.array([0] * draw).reshape(draw, 1)
                    ],
                    axis=1
                ).reshape(1, draw, 3)
            )
            for key in xarr.parameter.values
        },
        coords={
            "chain": (["chain"], np.arange(1)),
            "draw": (["draw"], np.arange(draw)),
            "a_dim": (["a_dim"], ["x", "y", "z"])

        }
    )
    
    xr_loglike = xr.Dataset(
        {
            'obs': (
                ['chain', 'draw', 'a_dim'],
                np.concatenate(
                    [
                        np.array([0] * draw).reshape(draw, 1),
                        np.array(xarr.log_like[1:]).reshape(draw, 1),
                        np.array([0] * draw).reshape(draw, 1)
                    ],
                    axis=1
                ).reshape(1, draw, 3)
            )
        },
        coords={
            "chain": (["chain"], np.arange(1)),
            "draw": (["draw"], np.arange(draw)),
            "a_dim": (["a_dim"], ["x", "y", "z"])

        }
    )
    return xr_prior, xr_post, xr_loglike

In [49]:
filename = get_most_recent_mod_output(POST_EST_DIR, 'mod4_rbc_vanilla')
xarr = xr.open_dataset(os.path.join(POST_EST_DIR, filename))
az_mod4 = az.InferenceData(**dict(zip(
    ['prior', 'posterior', 'log_likelihood'], 
    *[get_xarr_InferenceData(xarr, xarr.draw[-1].values)]
)))

filename = get_most_recent_mod_output(POST_EST_DIR, 'mod5_nk_vanilla')
xarr = xr.open_dataset(os.path.join(POST_EST_DIR, filename))
az_mod5 = az.InferenceData(**dict(zip(
    ['prior', 'posterior', 'log_likelihood'], 
    *[get_xarr_InferenceData(xarr, xarr.draw[-1].values)]
)))

In [51]:
az.compare({'mod4': az_mod4, 'mod5': az_mod5}, ic='loo')

c:\users\lukasgrahl\documents\git\macro_rbc\env_macro_rbc\lib\site-packages\arviz\stats\stats.py:1069: RuntimeWarning: overflow encountered in expm1
  x = np.expm1(-kappa * np.log1p(-probs)) / kappa
c:\users\lukasgrahl\documents\git\macro_rbc\env_macro_rbc\lib\site-packages\arviz\stats\stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
c:\users\lukasgrahl\documents\git\macro_rbc\env_macro_rbc\lib\site-packages\arviz\stats\stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
mod5,0,-203410.646053,202036.987805,0.000000,0.5,166084.097026,0.00000,True,log
mod4,1,-272037.309531,270990.279390,68626.663478,0.5,222117.533117,56033.43609,True,log


In [48]:
az.waic(az_mod4)

c:\users\lukasgrahl\documents\git\macro_rbc\env_macro_rbc\lib\site-packages\arviz\stats\stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 6821 posterior samples and 3 observations log-likelihood matrix.

          Estimate       SE
elpd_waic -1286198115.11  1050176363.38
p_waic    1286197068.08        -

There has been a warning during the calculation. Please check the results.

In [14]:
data2

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [17]:
xarr.log_like

<xarray.DataArray 'log_like' (draw: 4978)>
array([-312435.452538, -312435.452538, -312435.452538, ..., -312435.452538,
       -312435.452538, -312435.452538])
Coordinates:
  * draw     (draw) int32 0 1 2 3 4 5 6 7 ... 4971 4972 4973 4974 4975 4976 4977

In [4]:
filename = get_most_recent_mod_output(POST_EST_DIR, 'mod4_rbc_vanilla')
xarr = xr.open_dataset(os.path.join(POST_EST_DIR, filename))
prior, posterior = get_xarr_InferenceData(xarr, xarr.draw[-1].values)
az_mod4 = az.InferenceData(prior=prior, posterior=posterior)

filename = get_most_recent_mod_output(POST_EST_DIR, 'mod6_nk_energy_lin2')
xarr = xr.open_dataset(os.path.join(POST_EST_DIR, filename))
prior, posterior = get_xarr_InferenceData(xarr, xarr.draw[-1].values)
az_mod6 = az.InferenceData(prior=prior, posterior=posterior)

In [8]:
type(az_mod4)

arviz.data.inference_data.InferenceData

In [9]:
data1 = az.load_arviz_data("non_centered_eight")
data2 = az.load_arviz_data("centered_eight")


In [12]:
data2

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [11]:
az.loo(data2)

Computed from 2000 posterior samples and 8 observations log-likelihood matrix.

         Estimate       SE
elpd_loo   -30.78     1.35
p_loo        0.95        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)        6   75.0%
 (0.5, 0.7]   (ok)          2   25.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%

In [14]:
az_mod4

Inference data with groups:
	> posterior
	> prior

In [13]:
data1

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [12]:
compare_dict = {"non centered": az_mod4, "centered": az_mod6}
az.compare(compare_dict)

TypeError: Encountered error in ELPD computation of compare.